# <img src="https://github.com/JuliaLang/julia-logo-graphics/raw/master/images/julia-logo-color.png" height="100" /> _NCTSSOS Colab Notebook Template_

## Introduction NCTSSOS Julia
[NCTSSOS](https://github.com/wangjie212/NCTSSOS) is a non-commutative polynomial optimization tool based on the sparsity adapted moment-SOHS hierarchies.

[TSSOS](https://github.com/wangjie212/TSSOS) is a sparse polynomial optimization package based on the sparsity adapted moment-SOS hierarchies, which can fully exploit the sparsity in the problem data including correlative (variable) sparsity and term sparsity.

## Instructions
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. If you need a GPU: _Runtime_ > _Change runtime type_ > _Harware accelerator_ = _GPU_.
3. You might need to install Julia, IJulia and other packages (if needed, update `JULIA_VERSION` and the other parameters). This takes a couple of minutes.
4. Reload the page (press Ctrl+R, or ⌘+R, or the F5 key) after successfully installing Julia and continue to the next section.
5. Execute the following cell **Step 1-Step 7**(click on it and press Ctrl+Enter)

_Notes_:
* If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 5, 6 and 7.
* After installation, if you want to change the Julia version or activate/deactivate the GPU, you will need to reset the Runtime: _Runtime_ > _Factory reset runtime_ and repeat steps 5 and 6.


#__Step 1: Set up Colab Notebook__


## **1.1. Mount Drive - PY**


In [1]:
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# touch("/root/mosek/")
mv("/content/gdrive/MyDrive/Colab Notebooks/mosek.lic", "/root/mosek/")


## **1.2. Relocate MOSEK_lic**
Use the following code to move MOSEKLM_LICENSE_FILE to the default lic file search path ':/root/mosek/mosek.lic:'.

__* Set new_file_path as the place where you save MOSEK LICENSE__

In [2]:
import os
import shutil

new_folder_path = "/root/mosek"
os.makedirs(new_folder_path)

# Move

new_file_path = "/content/gdrive/MyDrive/Colab Notebooks/mosek.lic"
shutil.move(new_file_path , new_folder_path)

'/root/mosek/mosek.lic'

## Save tsv as txt

In [ ]:
import pandas as pd
import os
# Setting read route
directory = "/content/gdrive/MyDrive/Colab Notebooks/TSSOS/krebs_cycle/"
# Listing files under route
tsv_files = [f for f in os.listdir(directory) if f.endswith(".tsv")]
# Through every tsv
for tsv_file in tsv_files:
        df = pd.read_csv(os.path.join(directory, tsv_file), header = None, sep="\t", index_col = None)
        #df.insert(0, "Sample", tsv_file)
        txt_file = os.path.splitext(os.path.join(directory, tsv_file))[0] + ".txt"
        df.iloc[0,:100][1:].to_csv(txt_file, sep="\t", index=False, header = False)
        print(df.iloc[0,:10][1:])


LoadError: ArgumentError: Package pandas not found in current path.
- Run `import Pkg; Pkg.add("pandas")` to install the pandas package.

# __Step 2: Install Julia and pkgs__

## **2.1. Install Julia 1.8.2 and pkgs**

In [6]:
%%shell
set -e

JULIA_DEPOT_PATH="/content/gdrive/MyDrive/Colab Notebooks/test julia"
#---------------------------------------------------#
JULIA_VERSION="1.8.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools DelimitedFiles DynamicPolynomials Random Statistics DataFrames CSV"

JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.8.2 on the current Colab Runtime...
2024-04-20 10:10:24 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.8/julia-1.8.2-linux-x86_64.tar.gz [135859273/135859273] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package BenchmarkTools...
Installing Julia package DelimitedFiles...
Installing Julia package DynamicPolynomials...
Installing Julia package Random...
Installing Julia package Statistics...
Installing Julia package DataFrames...
Installing Julia package CSV...
Installing IJulia kernel...
[ Info: Installing julia kernelspec in /root/.local/share/jupyter/kernels/julia-1.8

Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then
jump to the 'Checking the Installation' section.


## **2.2. Notebook settings and check Julia version - JL**
__* Select Notebook setting: Set Edit-Notbook Setting-Runtime type-Julia__
* The `versioninfo()` function should print your Julia version and some other info about the system:

In [1]:
versioninfo()

Julia Version 1.8.2
Commit 36034abf260 (2022-09-29 15:21 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 2 × Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, broadwell)
  Threads: 2 on 2 virtual cores
Environment:
  LD_LIBRARY_PATH = /usr/local/nvidia/lib:/usr/local/nvidia/lib64
  JULIA_NUM_THREADS = 2


# __Step 3: Install Pkg_MOSEK__

For a more complete manual and full API reference, please refer to
[the MOSEK Julia API documentation](https://docs.mosek.com/latest/juliaapi/index.html).

## **3.1. Install**

Use the Julia package manager to install Mosek.jl, if not already installed:

```julia
Pkg.add("Mosek")
```

The `Mosek.jl` package requires the MOSEK distribution binaries run. Upon installation it will attempt to either local an installed MOSEK or download and install from the MOSEK website (www.mosek.com):



In [4]:
Pkg.add("Mosek")
using Pkg

Pkg.add("MosekTools")
using MosekTools

#Pkg.add(Pkg.PackageSpec(url="https://github.com/mosek/Mosek.jl",rev="master"))
Pkg.build("Mosek")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed Mosek ─ v10.1.4
    Updating `~/.julia/environments/v1.8/Project.toml`
  [6405355b] + Mosek v10.1.4
    Updating `~/.julia/environments/v1.8/Manifest.toml`
  [6405355b] + Mosek v10.1.4
    Building Mosek → `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/d745a2a4a0f71c4bcc8a3f5be3aff9feade3f644/build.log`
Precompiling project...
  ✓ Mosek
  1 dependency successfully precompiled in 8 seconds. 50 already precompiled.
   Resolving package versions...
   Installed Bzip2_jll ──────────── v1.0.8+1
   Installed CodecBzip2 ─────────── v0.8.2
   Installed IrrationalConstants ── v0.2.2
   Installed DiffRules ──────────── v1.15.1
   Installed DiffResults ────────── v1.1.0
   Installed StaticArraysCore ───── v1.4.2
   Installed SpecialFunctions ───── v2.3.1
   Installed NaNMath ────────────── v1.0.2
   Installed StaticArrays ───────── v1.9.3
   Installed CommonSubexpressions ─ v0.3

## Update the Mosek library*

If the MOSEK distro was installed manually, it can be updated simply by installing a newer distro in the same place. Otherwise, doing `Pkg.build("Mosek")` will check the latest MOSEK distro and update if possible.
You can see if the MOSEK distro was installed internally this way:

In [6]:
using Pkg
is_internal = open(joinpath(Pkg.dir("Mosek"),"deps","inst_method"),"r") do f read(f) == "internal" end

false

## Set environment*
Note that environment variables can be set temporarily from Julia as

In [ ]:
ENV["MOSEKBINDIR"] = "\\conten\\gdrive\\MyDrive\\Colab Notebooks\\TSSOS\\"

"\\conten\\gdrive\\MyDrive\\Colab Notebooks\\TSSOS\\"

# __Step 4: Install Pkg_TSSOS_NCTSSOS__

In [7]:
# Load the Pkg module
using Pkg

#import Pkgs TSSOS and NCTSSOS
Pkg.add(url="https://github.com/wangjie212/TSSOS")
Pkg.add(url="https://github.com/wangjie212/NCTSSOS")

     Cloning git-repo `https://github.com/wangjie212/TSSOS`
    Updating git-repo `https://github.com/wangjie212/TSSOS`
   Resolving package versions...
   Installed DynamicPolynomials ────── v0.4.6
   Installed MUMPS_seq_jll ─────────── v5.4.1+0
   Installed MultivariatePolynomials ─ v0.4.7
   Installed QDLDL ─────────────────── v0.4.1
   Installed JLD2 ──────────────────── v0.4.46
   Installed IterTools ─────────────── v1.10.0
   Installed ArnoldiMethod ─────────── v0.4.0
   Installed Inflate ───────────────── v0.1.4
   Installed Ipopt ─────────────────── v1.6.2
   Installed ChordalGraph ──────────── v0.4.1
   Installed COSMO ─────────────────── v0.8.8
   Installed COSMOAccelerators ─────── v0.1.0
   Installed Dualization ───────────── v0.5.8
   Installed FileIO ────────────────── v1.16.3
   Installed Ipopt_jll ─────────────── v300.1400.400+0
   Installed OpenBLAS32_jll ────────── v0.3.17+0
   Installed SemialgebraicSets ─────── v0.2.5
   Installed UnsafeArrays ──────────── v1.0.5
  

## Check for package path*

In [8]:
# Load the Pkg module
using Pkg

# Replace "PackageName" with the name of the package you want to find the path for
package_name = "DelimitedFiles"

# Find the path of the installed package
path_to_mosek = Base.find_package(package_name)

# Display the path
println("Path of $package_name: $path_to_mosek")


Path of DelimitedFiles: /usr/local/share/julia/stdlib/v1.8/DelimitedFiles/src/DelimitedFiles.jl


# __Step 5: Test for Krebs cycle__
* Read Krebs_cycle tsv Observations
* NCTSSOS
* Runtime Plot

## **5.1. Try to active pkgs**
If any errors happen, check for Julia and pkgs installation and go back to Step 1-4.

In [10]:
using NCTSSOS
using TSSOS
using DynamicPolynomials
using CSV
using Random
using Statistics
using DataFrames
using DelimitedFiles

## **5.2. Read Krebs_cycle tsv Observations**
* To execute the notebook directly in colab make sure your tsv files are in the location /krebs_cycle inside this notebook's internal filesystem.
* Data discription: 10 tsv files about krebs_cycle time series data have been collected.

In [11]:
# the stock-market.txt is generated by
"""
# Load stock-market data
load_path = 'setting6.mat'
load_data = sio.loadmat(load_path)
seq=flatten(load_data['seq_d0'].tolist())
"""
ts = readdlm( "/content/gdrive/MyDrive/Colab Notebooks/TSSOS/krebs_cycle/series1713271727120.txt" );

# __Step 6: TSSOS_Krebs cycle__
* Find a locally optimal solution and predicted Y
* Functions(observation, parameter_estimation and rmse)

In [12]:
function observation(T,ts,start)
    # randomly select a trajectory of time series
    return copy(ts[start:start+T-1])
end


function parameter_estimation(Y)
    # training process
    # use Y to learn the system parameters in f[t+1] = Fdash*f[t] + p[t]
    # output Fdash and p[1:T]

    T=length(Y)

    @polyvar G p[1:T] q[1:T-1] f[1:T+1] m[1:T+1]; # all variables are assumed to be nc
    var=vcat(m,f,p,q,G);

    # constraints
    ine1 = [f[i] - m[i] - p[i] for i in 1:T];
    ine2 = [- f[i] + m[i] + p[i] for i in 1:T];
    ine3 = [m[i+1] - G*m[i] - q[i] for i in 1:T-1];
    ine4 = [-m[i+1] + G*m[i] + q[i] for i in 1:T-1];
    ine5 = [m[T+1] - G*m[T],-m[T+1] + G*m[T]]
    #ine6 = [f[T+1] - m[T+1],-f[T+1] + F*m[T+1]]

    #objective
    obj=sum((Y[i]-f[i])^2 for i in 1:T)+ 0.01*sum(p[i]^2 for i in 1:T) +0.01*sum(q[i]^2 for i in 1:T-1) #+ p[T+1]^2 + q[T]^2 #+ 0.0001*sum(p[i]^2 for i in 1:T); #+0.1*q[i]^2

    # pop
    pop=vcat(obj,ine1,ine2,ine3,ine4,ine5); #,ine5,ine6

    # solve model
    opt,sol,data=tssos_first(pop,var,1,TS="MD",solution=true);
    print(sol)
    return sol[T+1]
end


function rmse(Y_predict, Y_true)
    # compare predicted Y_[T+1：T+pred] with actual value of predicted Y_[T+1：T+pred]
    # calcluate rmse: sqrt( sum((Y-f)^2)/N)
    Y=copy(Y_true[1:length(Y_predict)])
    return  abs(Y[1]-Y_predict[1])
    #1-sum( abs(Y[i]-Y_predict[i]) for i in 1:length(Y_predict) )/ sum(abs(Y[i]-mean(Y)) for i in 1:length(Y_predict) )
end

rmse (generic function with 1 method)

## **6.1. Parameter estimations**

In [16]:
## a single run (because the first time using TSSOS is very slow)
# the length of time window is 20
T=20
# we are making one-step ahead prediction
pred=1
# select a 21-period time series from Krebs cycle data.
Y=observation(T+pred,ts,1)
# system identification using the first 20-period
parameter_estimation(Y[1:T])


*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2024
TSSOS is launching...
Starting to compute the block structure...
-----------------------------------------------------------------------------
The sizes of PSD blocks:
[3, 2, 1]
[20, 60, 1]
-----------------------------------------------------------------------------
Obtained the block structure in 0.010734857 seconds.
The maximal size of blocks is 3.
Assembling the SDP...
There are 3486 affine constraints.
SDP assembling time: 2.303642723 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 3486            
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 82              
  Matrix variables     

45.152258186725426

## **6.2. RMSE**

In [ ]:
T=20
pred=1

opt=DataFrame(time=[],pred=[],rmse=[])
for s in 1:100
    Y=observation(T+pred,ts,s);
    Y_predict=parameter_estimation(Y[1:T]);
    opt_tem=rmse(Y_predict,Y[T+1:T+pred]); # assess the prediction
    push!(opt,(s,Y_predict,opt_tem))
end

*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2024
TSSOS is launching...
Starting to compute the block structure...
-----------------------------------------------------------------------------
The sizes of PSD blocks:
[3, 2, 1]
[20, 60, 1]
-----------------------------------------------------------------------------
Obtained the block structure in 0.008883991 seconds.
The maximal size of blocks is 3.
Assembling the SDP...
There are 3486 affine constraints.
SDP assembling time: 2.689908021 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 3486            
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 82              
  Matrix variables     

## **6.3. Save Krebs cycle TSSOS test results**

In [ ]:
CSV.write("Krebs_cycle_TSSOS_d1.csv", opt)
opt

# __Step 7: NCTSSOS_Krebs cycle__
* Function timing

In [ ]:
function timing(T,level)
    #T=length(Y)
    start=rand(0:100)
    Y=ts[start+1:start+T]

    @ncpolyvar G Fdash p[1:T] q[1:T] f[1:T+1] m[1:T+1]; # all variables are assumed to be nc
    var=vcat(m,f,p,q,G,Fdash);

    # constraints
    ine1 = [f[i] - Fdash*m[i] - p[i] for i in 1:T];
    ine2 = [- f[i] + Fdash*m[i] + p[i] for i in 1:T];
    ine3 = [m[i+1] - G*m[i] - q[i] for i in 1:T];
    ine4 = [-m[i+1] + G*m[i] + q[i] for i in 1:T];

    #objective
    obj=sum((Y[i]-f[i])^2 for i in 1:T)+ 0.01*sum(p[i]^2 for i in 1:T) +0.01*sum(q[i]^2 for i in 1:T) #+ p[T+1]^2 + q[T]^2 #+ 0.0001*sum(p[i]^2 for i in 1:T); #+0.1*q[i]^2

    # pop
    pop=[obj;vcat(ine1,ine2,ine3,ine4)]

    # solve model
    tssos_first = nctssos.nctssos_first
    return @elapsed nctssos_first(pop,var,level,numeq=0,TS="MD", obj="trace")
end

timing (generic function with 1 method)

## **7.1. Save NCTSSOS runtime**

In [ ]:
function runtime_NCTSSOS()
    for level in 1:2
        meanO=Float64[]
        stdO=Float64[]
        for T in 5:30
            rtime=Float64[]
            for r in 1:3
                run_tem=timing(T,level)
                print(run_tem)
                push!(rtime,copy(run_tem))
            end
            push!(meanO,mean(copy(rtime)))
            push!(stdO,std(copy(rtime)))
        end
        writedlm( string("krebs_cycle_NCTSSOS_k",level,".csv"),hcat(collect(5:30),meanO,stdO), ',')
    end
    writedlm( "Krebs_cycle_opt.csv", opt, ',')
end
runtime_NCTSSOS()

LoadError: MethodError: no method matching *(::Nothing, ::Vector{UInt16})
[0mClosest candidates are:
[0m  *(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:591
[0m  *([91m::StridedMatrix{T}[39m, ::StridedVector{S}) where {T<:Union{Float32, Float64, ComplexF32, ComplexF64}, S<:Real} at /usr/local/share/julia/stdlib/v1.8/LinearAlgebra/src/matmul.jl:49
[0m  *([91m::StridedMatrix{var"#s31"} where var"#s31"<:MutableArithmetics.AbstractMutable[39m, ::StridedVector{var"#s37"} where var"#s37") at ~/.julia/packages/MutableArithmetics/umkRQ/src/dispatch.jl:359
[0m  ...

In [ ]:
writedlm( string("Krebs_cycle_NCTSSOS_k",2,".csv"),hcat(collect(5:24),meanO,stdO), ',')

LoadError: UndefVarError: meanO not defined

# <img src="https://github.com/JuliaLang/julia-logo-graphics/raw/master/images/julia-logo-color.png" height="60" /> Need Help?

* Learning: https://julialang.org/learning/
* Documentation: https://docs.julialang.org/
* Questions & Discussions:
  * https://discourse.julialang.org/
  * http://julialang.slack.com/
  * https://stackoverflow.com/questions/tagged/julia

If you ever ask for help or file an issue about Julia, you should generally provide the output of `versioninfo()`.

In [ ]:
import pandas as pd
import os

# 原始文件位置
source_path = "/content/gdrive/MyDrive/Colab Notebooks/TSSOS/krebs_cycle/"
# 保存位置
save_path = "/content/gdrive/MyDrive/Colab Notebooks/TSSOS/krebs_cycle/"
if not os.path.exists(save_path):
    os.mkdir(save_path)
pathDir = os.listdir(source_path)
Name = []
End = []


# 获得文件的名称和后缀
def getName(workdir):
    for filename in os.listdir(workdir):
        split_file = os.path.splitext(filename)
        # print(split_file[0])
        Name.append(split_file[0])
        End.append(split_file[1])
    return Name, End


name, end = getName(source_path)
# print(Name, End)

TsvFile = os.listdir(source_path)
# print(len(TsvFile))
# print(TsvFile)

# 循环将tsv文件转为csv文件
for long in range(len(TsvFile)):
    with open(source_path + TsvFile[long], 'r', encoding='utf-8') as tsv_file:
        # print(tsv_file)
        if end[long] == '.tsv':
            pd_all = pd.read_csv(tsv_file, sep='\t')
            pd_all.to_csv(save_path + name[long] + '.csv', index=False, sep=',')

CsvFile = os.listdir(save_path)
# print(len(CsvFile))
# print(CsvFile)

# 循环合并csv文件
# f2是我的label文件
for long in range(len(CsvFile)):
    f2 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/TSSOS/krebs_cycle/series1713272059020.csv')
    with open(save_path + CsvFile[long], 'r', encoding='utf-8') as csv_file:
        # print(csv_file)
        f1 = pd.read_csv(csv_file)
        file = [f1, f2]
        # print(file)
        # axis=1 列合并
		# axis=0 行合并（默认）
        train = pd.concat(file, axis=1)
        train.to_csv(save_path + name[long] + '.csv', index=False, sep=',')

